## Setting up imports. CRDS_CACHE is a directory containing calibration files for JWST. Can be downloaded from the JWST Python library page, also there on the correct folder on vcompute

### Setting up additional directory structure:

1. Create three more folders in addition to CRDS_CACHE inside the root folder. This script should be in the root folder. Name these S3D, X1D and Newcals. S3D will contain the final 3D spectrum cubes. X1D will contain the 1D spectra of auto-detected point sources -- not relevant to us. Newcals will contain the intermediate Level 2 files we produce from each frame.

2. Download the \_calints.fits and \_cal.fits files from MAST for the April 6 observation. Place these in the root folder. 

3. Download any one \_asn.json file from the five segment data products. I downloaded the one in seg003. The "name" parameter in it gives the file name of the final output (3D spectral cubes). Change the name to any distinguishable string. I used "hello". We'll replace this with the identifier numbers inside the code. Place this in the root folder as well.

In [1]:
import os
os.environ['CRDS_PATH'] = 'CRDS_CACHE'
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'

from jwst.pipeline import Spec2Pipeline, Spec3Pipeline, Tso3Pipeline
from jwst.residual_fringe import ResidualFringeStep

from astropy.io import fits

### In the single exposure of 6 April, there are 85 integrations. To avoid large file sizes, all 85 integrations are not in a single file. They are split into 5 separate "SEGMENT" files. First 4 segments have 18 files, the last one has 13.

## This is the main cell. Here is what it does, line-by-line (follow line numbers in comments next to code):

1. for loop iterates over the five segments.
2. We open the calints file (with integration level data, shape 18 x 1024 x 1032, and the corresponding stacked cal file, shape 1024 x 1032. The pipeline needs the cal file to work.
3. Segments 1 to 4 have 18 integrations each. Segment 5 has only 13 integrations. We set the ending index for the integration loop accordingly.
4. The new for loop creates a cal file for each integration:  
   (a) Set the starting and ending integration number to the current integration number, j  
   (b) Set the data array (ff2[1]) and the error array (ff2[2]) to the j-th data array and error array from the Calints file  
   (c) Write this out as a new cal fits file. Since these are temporary, the file name doesn't really matter.
5. The next for loop creates an asn.json file for each of the new cal files. These json files do not have any use by themselves, but are needed to be fed into the pipeline in that format. The entire loop is just file opening, editing and writing.
6. The next loop is where the main reduction occurs. Each new cal file, now containing a single integration instead of a stack, is processed to a 3D cube (2 spatial, 1 spectral) and saved as an S3D file. The loop only calls the Spec3Pipeline on the asn.json files, which in turn have been written on with the names of the respective cal files.
7. Once the S3D files are created, we need to assign them a unique identification file name. I used the format {date}\_{exposure\_number}\_{segment\_number}\_final\_{integration\_number}\_{channel\_number}\_short_s3d.fits. Of this, the segment number and integration numbers and channels are taken care of the for loop. The date and exposure numbers need to be manually entered, for example here 9_2
8. Send the S3D (and the auxiliary X1D) files into their respective folders with the correct names.

In [7]:
for ctr in range(5):    # Line 1
    ff1 = fits.open('jw04572009001_02101_00001-seg00{}_mirifushort_calints.fits'.format(ctr+1))    # Line 2
    ff2 = fits.open('jw04572009001_02101_00001-seg00{}_mirifushort_cal.fits'.format(ctr+1))    #Line 2

    end = 19    # Line 3
    if ctr == 4:   # Line 3
        end = 14    # Line 3
    for j in range(ctr*18+1, ctr*18+end):    # Line 4
        ff2[0].header['INTSTART'] = j     # Line 4-a
        ff2[0].header['INTEND'] = j       # Line 4-a
    
        ff2[1].data = ff1[1].data[j- ctr*18 -1,:,:]    # Line 4-b
        ff2[2].data = ff1[2].data[j- ctr*18 -1,:,:]    # Line 4-b
        ff2.writeto('Newcals/new_{}_cal.fits'.format(j- ctr*18 -1), overwrite=True)    # Line 4-c

    name = 'jw04572009001_02101_00001-seg003_mirifushort_'    # Line 5
    for j in range(37,55):    # Line 5
        a = 'jw04572-o009_20240714t145526_tso-spec2_00005_asn.json'    # Line 5
        wr = open(a)    # Line 5
        s = wr.read()    # Line 5
        a = 'Newcals/new_{}_asn.json'.format(j-37)    # Line 5
        wr = open(a, 'w+')    # Line 5
        #name = r.rstrip('rate.fits')
        s = s.replace(name+'rateints.fits' , 'new_{}_cal.fits'.format(j-37))    # Line 5
        s = s.replace('hello', 'final_{}'.format(j-37))    # Line 5
        wr.write(s)    # Line 5
        wr.close()    # Line 5

    last = 18    # Line 6
    if ctr == 4:
        last = 13    # Line 6
    for j in range(last):    # Line 6
        print('\n\n\n\n',j,'\n\n\n\n')    # Line 6
        result = Spec3Pipeline.call('Newcals/new_{}_asn.json'.format(j))    # Line 6

    
    pred = '6_2_seg00{}_'.format(ctr+1)      # Line 7
    files = [f for f in os.listdir('.') if os.path.isfile(f)]      # Line 8
    for f in files:      # Line 8
        if 's3d.fits' in f:      # Line 8
            os.rename(f, 'S3D/'+pred+f)      # Line 8
        if 'x1d.fits' in f:      # Line 8
            os.rename(f, 'X1D/'+pred+f)      # Line 8







 0 






2024-07-15 15:04:24,352 - stpipe - INFO - PARS-OUTLIERDETECTIONSTEP parameters found: CRDS_CACHE/references/jwst/miri/jwst_miri_pars-outlierdetectionstep_0052.asdf
2024-07-15 15:04:24,364 - stpipe - INFO - PARS-RESAMPLESPECSTEP parameters found: CRDS_CACHE/references/jwst/miri/jwst_miri_pars-resamplespecstep_0001.asdf
2024-07-15 15:04:24,377 - CRDS - ERROR -  Error determining best reference for 'pars-spectralleakstep'  =   Unknown reference type 'pars-spectralleakstep'
2024-07-15 15:04:24,379 - stpipe - INFO - PARS-SPEC3PIPELINE parameters found: CRDS_CACHE/references/jwst/miri/jwst_miri_pars-spec3pipeline_0005.asdf
2024-07-15 15:04:24,403 - stpipe.Spec3Pipeline - INFO - Spec3Pipeline instance created.
2024-07-15 15:04:24,404 - stpipe.Spec3Pipeline.assign_mtwcs - INFO - AssignMTWcsStep instance created.
2024-07-15 15:04:24,406 - stpipe.Spec3Pipeline.master_background - INFO - MasterBackgroundStep instance created.
2024-07-15 15:04:24,407 - stpipe.Spec3Pipeline.mrs_imatch - INFO - MRSI





 1 






2024-07-15 15:04:54,976 - stpipe - INFO - PARS-OUTLIERDETECTIONSTEP parameters found: CRDS_CACHE/references/jwst/miri/jwst_miri_pars-outlierdetectionstep_0052.asdf
2024-07-15 15:04:54,986 - stpipe - INFO - PARS-RESAMPLESPECSTEP parameters found: CRDS_CACHE/references/jwst/miri/jwst_miri_pars-resamplespecstep_0001.asdf
2024-07-15 15:04:54,997 - CRDS - ERROR -  Error determining best reference for 'pars-spectralleakstep'  =   Unknown reference type 'pars-spectralleakstep'
2024-07-15 15:04:54,998 - stpipe - INFO - PARS-SPEC3PIPELINE parameters found: CRDS_CACHE/references/jwst/miri/jwst_miri_pars-spec3pipeline_0005.asdf
2024-07-15 15:04:55,014 - stpipe.Spec3Pipeline - INFO - Spec3Pipeline instance created.
2024-07-15 15:04:55,015 - stpipe.Spec3Pipeline.assign_mtwcs - INFO - AssignMTWcsStep instance created.
2024-07-15 15:04:55,016 - stpipe.Spec3Pipeline.master_background - INFO - MasterBackgroundStep instance created.
2024-07-15 15:04:55,018 - stpipe.Spec3Pipeline.mrs_imatch - INFO - MRSI





 2 






...

2024-07-15 15:44:46,835 - stpipe - INFO - PARS-OUTLIERDETECTIONSTEP parameters found: CRDS_CACHE/references/jwst/miri/jwst_miri_pars-outlierdetectionstep_0052.asdf
2024-07-15 15:44:46,846 - stpipe - INFO - PARS-RESAMPLESPECSTEP parameters found: CRDS_CACHE/references/jwst/miri/jwst_miri_pars-resamplespecstep_0001.asdf
2024-07-15 15:44:46,859 - CRDS - ERROR -  Error determining best reference for 'pars-spectralleakstep'  =   Unknown reference type 'pars-spectralleakstep'
2024-07-15 15:44:46,861 - stpipe - INFO - PARS-SPEC3PIPELINE parameters found: CRDS_CACHE/references/jwst/miri/jwst_miri_pars-spec3pipeline_0005.asdf
2024-07-15 15:44:46,880 - stpipe.Spec3Pipeline - INFO - Spec3Pipeline instance created.
2024-07-15 15:44:46,881 - stpipe.Spec3Pipeline.assign_mtwcs - INFO - AssignMTWcsStep instance created.
2024-07-15 15:44:46,883 - stpipe.Spec3Pipeline.master_background - INFO - MasterBackgroundStep instance created.
2024-07-15 15:44:46,884 - stpipe.Spec3Pipeline.mrs_imatch - INFO - MRSI





 11 






2024-07-15 15:45:16,186 - stpipe - INFO - PARS-OUTLIERDETECTIONSTEP parameters found: CRDS_CACHE/references/jwst/miri/jwst_miri_pars-outlierdetectionstep_0052.asdf
2024-07-15 15:45:16,196 - stpipe - INFO - PARS-RESAMPLESPECSTEP parameters found: CRDS_CACHE/references/jwst/miri/jwst_miri_pars-resamplespecstep_0001.asdf
2024-07-15 15:45:16,207 - CRDS - ERROR -  Error determining best reference for 'pars-spectralleakstep'  =   Unknown reference type 'pars-spectralleakstep'
2024-07-15 15:45:16,209 - stpipe - INFO - PARS-SPEC3PIPELINE parameters found: CRDS_CACHE/references/jwst/miri/jwst_miri_pars-spec3pipeline_0005.asdf
2024-07-15 15:45:16,224 - stpipe.Spec3Pipeline - INFO - Spec3Pipeline instance created.
2024-07-15 15:45:16,226 - stpipe.Spec3Pipeline.assign_mtwcs - INFO - AssignMTWcsStep instance created.
2024-07-15 15:45:16,227 - stpipe.Spec3Pipeline.master_background - INFO - MasterBackgroundStep instance created.
2024-07-15 15:45:16,228 - stpipe.Spec3Pipeline.mrs_imatch - INFO - MRSI





 12 






2024-07-15 15:45:45,683 - stpipe - INFO - PARS-OUTLIERDETECTIONSTEP parameters found: CRDS_CACHE/references/jwst/miri/jwst_miri_pars-outlierdetectionstep_0052.asdf
2024-07-15 15:45:45,693 - stpipe - INFO - PARS-RESAMPLESPECSTEP parameters found: CRDS_CACHE/references/jwst/miri/jwst_miri_pars-resamplespecstep_0001.asdf
2024-07-15 15:45:45,704 - CRDS - ERROR -  Error determining best reference for 'pars-spectralleakstep'  =   Unknown reference type 'pars-spectralleakstep'
2024-07-15 15:45:45,705 - stpipe - INFO - PARS-SPEC3PIPELINE parameters found: CRDS_CACHE/references/jwst/miri/jwst_miri_pars-spec3pipeline_0005.asdf
2024-07-15 15:45:45,722 - stpipe.Spec3Pipeline - INFO - Spec3Pipeline instance created.
2024-07-15 15:45:45,723 - stpipe.Spec3Pipeline.assign_mtwcs - INFO - AssignMTWcsStep instance created.
2024-07-15 15:45:45,724 - stpipe.Spec3Pipeline.master_background - INFO - MasterBackgroundStep instance created.
2024-07-15 15:45:45,725 - stpipe.Spec3Pipeline.mrs_imatch - INFO - MRSI

#### Cell for Sgr A* coordinates. Not relevant here

In [1]:
ra = '17 45 40.0360'
s = [float(i) for i in ra.split()]
ra = s[0]*15 + s[1]/4 + s[2]/240

dec = '-29 00 28.17'
s = [float(i) for i in dec.split()]
dec = s[0] - s[1]/60 - s[2]/3600

ra, dec

(266.41681666666665, -29.007825)